In [1]:
all_locations = {
    "1. Istanbul, Turkey: A city that straddles Europe and Asia, known for its rich history and cultural landmarks like Hagia Sophia, a masterpiece blending Byzantine and Ottoman architecture; the Blue Mosque with its iconic minarets and blue tiles; Topkapi Palace, home to Ottoman sultans for centuries; the bustling Grand Bazaar, one of the world’s oldest markets; and the Bosphorus Cruise, offering scenic views of palaces, bridges, and waterfront mansions.": [41.0082, 28.9784],
    
    "2. Cappadocia, Turkey: Famous for its surreal landscapes filled with unique fairy chimneys, cave dwellings, and rock formations. It is a top destination for hot air balloon rides at sunrise, offering unforgettable panoramic views of valleys such as Göreme and Devrent. The region also boasts ancient underground cities like Derinkuyu and Kaymaklı, used as hiding places during Byzantine times.": [38.6431, 34.8309],
    
    "3. Pamukkale, Turkey: Known as the 'Cotton Castle' for its white travertine terraces formed by thermal waters cascading down the hills. Visitors can soak in the warm, mineral-rich pools with stunning views. The ancient city of Hierapolis, perched atop Pamukkale, features well-preserved Roman ruins, including a vast necropolis, a theater, and the famous Cleopatra’s Pool, said to have healing properties.": [37.9244, 29.1187],
    
    "4. Ephesus, Turkey: One of the best-preserved ancient cities in the Mediterranean, showcasing Roman and Greek architecture. Highlights include the Library of Celsus, a grand ancient library with intricate carvings; the massive Great Theater capable of seating 25,000 spectators; and the ruins of the Temple of Artemis, once one of the Seven Wonders of the Ancient World.": [37.9392, 27.3414],
    
    "5. Antalya, Turkey: A popular resort city on the Turkish Riviera, known for its stunning beaches like Konyaaltı and Lara, turquoise waters, and luxury resorts. The charming Old Town (Kaleiçi) features narrow cobblestone streets, Ottoman mansions, and historic landmarks. Nearby attractions include the Düden Waterfalls and ancient ruins like Perge and Aspendos with its impressive Roman theater.": [36.8969, 30.7133],
    
    "6. Bodrum, Turkey: A coastal town renowned for its beaches, nightlife, and historical significance. It is home to the Bodrum Castle, built by the Knights Hospitaller and now housing the Museum of Underwater Archaeology. The town also contains the remains of the Mausoleum at Halicarnassus, one of the Seven Wonders of the Ancient World. Sailing along the Aegean coast is a favorite activity here.": [37.0344, 27.4305],
    
    "7. Troy, Turkey: The legendary city of the Trojan War described in Homer’s Iliad. Archaeological excavations reveal layers of ancient cities built on top of one another, dating back to the Bronze Age. Visitors can see the symbolic replica of the Trojan Horse and ruins of city walls, temples, and houses, offering a glimpse into one of history’s most famous mythological battles.": [39.9578, 26.2389],
    
    "8. Mount Ararat, Turkey: The highest peak in Turkey at 5,137 meters, believed to be the resting place of Noah’s Ark according to biblical tradition. It is a prominent destination for mountaineering and trekking enthusiasts, offering challenging ascents and breathtaking views of the surrounding landscapes near the Turkish-Iranian border.": [39.7022, 44.2983],
    
    "9. Ankara, Turkey: The capital city of Turkey, blending modernity with history. Key attractions include Anıtkabir, the monumental mausoleum of Mustafa Kemal Atatürk, the founder of modern Turkey. The Museum of Anatolian Civilizations houses artifacts from ancient civilizations, while the old citadel and Roman ruins showcase the city's deep historical roots.": [39.9334, 32.8597],
    
    "10. Trabzon, Turkey: Located on the Black Sea coast, Trabzon is known for its lush green landscapes and cultural landmarks. The cliffside Sumela Monastery, built into the rocks of the Pontic Mountains, offers stunning views and historical significance. Trabzon is also famous for its tea plantations, local cuisine, and historic sites like the Hagia Sophia of Trabzon.": [41.0015, 39.7178],
    
    "11. Gallipoli Peninsula, Turkey: A site of immense historical importance, particularly for the Gallipoli Campaign of World War I. It is home to numerous memorials, cemeteries, and battlefields commemorating the soldiers who fought and perished during the campaign. Visitors can explore landmarks like Anzac Cove, Lone Pine Cemetery, and Chunuk Bair.": [40.3675, 26.4503],
    
    "12. Fethiye, Turkey: A stunning coastal town known for its turquoise waters, beautiful beaches, and vibrant harbor. It is home to the famous Ölüdeniz Blue Lagoon, perfect for swimming and paragliding, offering dramatic views of the coastline. Nearby attractions include the ghost town of Kayaköy, the Lycian Way hiking trail, and the ancient ruins of Telmessos.": [36.6210, 29.1163]
}


In [2]:
import folium
import geopandas as gpd
import pycountry
from folium import IFrame

def create_auto_centered_map(locations, output_file="map_route.html"):
    # Initialize map without specifying center and zoom
    auto_center_map = folium.Map()

    # Add all locations to the map
    for location, coordinates in locations.items():
        # Split location name and description
        title, description = location.split(": ", 1)
        
        # Generate link to corresponding .md file
        # Create HTML content for popup with short description and link
        html_content = f"""
        <div style="font-family: Arial, sans-serif; font-size: 12px;">
            <b>{title}</b><br>
            {description.split('.')[0]}.<br>
        </div>
        """
        iframe = IFrame(html_content, width=250, height=150)  # Adjust popup size

        # Add marker with formatted popup
        folium.Marker(
            location=coordinates,
            popup=folium.Popup(iframe),
        ).add_to(auto_center_map)

    # Add edges from i to i+1
    locations_list = list(locations.values())
    for i in range(len(locations_list) - 1):
        folium.PolyLine(locations_list[i:i+2], color="blue", weight=2.5, opacity=1).add_to(auto_center_map)

    # Save map to HTML file
    auto_center_map.save(output_file)
    print(f"Map has been saved to {output_file}")

def highlight_countries_on_map(countries, shapefile_path, output_file="highlighted_countries_map.html"):
    # Load the world shapefile from the local file
    world = gpd.read_file(shapefile_path)

    # Convert country names to ISO Alpha-3 codes using pycountry
    country_codes = []
    for country in countries:
        try:
            country_code = pycountry.countries.lookup(country).alpha_3
            country_codes.append(country_code)
        except LookupError:
            print(f"Warning: '{country}' not found in pycountry database.")

    # Filter for the specified countries by ISO code
    highlighted = world[world["ADM0_A3"].isin(country_codes)]

    # Initialize a folium map centered globally
    world_map = folium.Map(location=[20, 0], zoom_start=2)

    # Add highlighted countries to the map in red
    folium.GeoJson(
        highlighted,
        style_function=lambda x: {'fillColor': 'red', 'color': 'black', 'weight': 0.5, 'fillOpacity': 0.7}
    ).add_to(world_map)

    # Save the map to an HTML file
    world_map.save(output_file)
    print(f"Map has been saved to {output_file}")

# # Define all locations with coordinates and labels
# all_locations = {
#     # South China
#     "1. Hong Kong": [22.3193, 114.1694],
#     "2. Shenzhen, China": [22.5431, 114.0579],
#     "3. Guangzhou, China": [23.1291, 113.2644],
#     # South-Central China
#     "4. Nanning, China": [22.8170, 108.3669],
#     "5. Guilin, China": [25.2744, 110.2900],
#     "6. Kunming, China": [24.8801, 102.8329],
#     # Yunnan Province Cluster
#     "7. Dali, China": [25.6939, 100.1619],
#     "8. Lijiang, China": [26.8721, 100.2257],
#     "9. Shangri-La, China": [27.8252, 99.7074],
#     # Central and Southwest China
#     "10. Chongqing, China": [29.5630, 106.5516],
#     "11. Chengdu, China": [30.5728, 104.0668],
#     # North-Central China
#     "12. Xi’an, China": [34.3416, 108.9398],
#     "13. Luoyang, China": [34.6587, 112.4249],       # Henan, near Xi'an
#     "14. Zhengzhou, China": [34.7466, 113.6254],     # Henan, capital of Henan
#     "15. Kaifeng, China": [34.7972, 114.3075],       # Henan, close to Zhengzhou
#     "16. Anyang, China": [36.0976, 114.3924],
#     "17. Pingyao, China": [37.1987, 112.1784],
#     "18. Datong, China": [40.0768, 113.3001],
#     "19. Mount Wutai, China": [39.0585, 113.5931],
#     "20. Shijiazhuang, China": [38.0428, 114.5149],
#     # Beijing to Shanghai Route
#     "21. Beijing, China": [39.9042, 116.4074],
#     "22. Qufu, China": [35.5964, 116.9919],
#     "23. Mount Tai, China": [36.2506, 117.1018],
#     "24. Qingdao, China": [36.0671, 120.3826],
#     "25. Nanjing, China": [32.0603, 118.7969],
#     "26. Shanghai, China": [31.2304, 121.4737],
#     "27. Fuzhou, China": [26.0765, 119.2911],
#     "28. Xiamen, China": [24.4798, 118.0894],
#     # West China (Xinjiang)
#     "29. Urumqi, China": [43.8256, 87.6168],
#     "30. Turpan, China": [42.9513, 89.1897],
#     "31. Kashgar, China": [39.4704, 75.9898],
#     "32. Mount Kailash, Tibet": [31.0685, 81.3104],
#     "33. Lake Manasarovar, Tibet": [30.6476, 81.4634],
#     "34. Darchen, Tibet": [30.9650, 81.3189],
#     "35. Pulan County (Burang), Tibet": [30.2972, 81.2464],
#     "36. Hilsa, Nepal": [30.0381, 81.1750],
#     "37. Lhasa, China (Tibet)": [29.6520, 91.1721],  # Tibet's capital
#     "38. Shigatse, China (Tibet)": [29.2682, 88.8806],  # Major city near Everest
#     "39. Kathmandu, Nepal": [27.7172, 85.3240],  # Nepal's capital
#     "40. Lumbini, Nepal": [27.4695, 83.2750],
#     "41. Varanasi, India": [25.3176, 82.9739],
#     "42. Agra, India": [27.1751, 78.0421],  # Taj Mahal location
#     "43. Jaipur, India": [26.9124, 75.7873],
#     "44. Delhi, India": [28.6139, 77.2090],  # India’s capital
# }

create_auto_centered_map(all_locations)

# List of countries to highlight
countries_to_highlight = ["China", "Mexico", "United States", "Canada", "Guatemala", "Honduras", "Spain", "United Kingdom"]

# Call the function to highlight countries and save to HTML
highlight_countries_on_map(countries_to_highlight, shapefile_path="/Users/songye03/Desktop/me/ne_110m_admin_0_countries/ne_110m_admin_0_countries.shp")

Map has been saved to map_route.html
Map has been saved to highlighted_countries_map.html


In [3]:
# Generate the HTML content
html_content = """
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Travel Locations</title>
    <style>
        body { font-family: Arial, sans-serif; line-height: 1.6; margin: 2em; }
        h1 { text-align: center; }
        ul { list-style-type: none; padding: 0; }
        li { margin-bottom: 1em; }
        a { text-decoration: none; color: #007BFF; }
        a:hover { text-decoration: underline; }
    </style>
</head>
<body>
    <h1>Travel Locations</h1>
    <ul>
"""

# Add each location with its description and hyperlink
for location in all_locations:
    title, description = location.split(": ", 1)
    # Update the city_name links to point to blog posts
    city_name = " ".join(title.split(". ", 1)[-1].split(",")[0].strip().split()).lower().replace(" ", "-")
    html_content += f"""
        <li>
            <strong>{title.split(". ", 1)[-1]}</strong>: {description}
            <br>
            <a href="/{city_name}/" target="_blank">Read more</a>
        </li>
    """

# Close the HTML tags
html_content += """
    </ul>
</body>
</html>
"""

# Save the HTML content to a file
output_file = "locations_list.html"
with open(output_file, "w", encoding="utf-8") as file:
    file.write(html_content)

print(f"HTML file has been created: {output_file}")

HTML file has been created: locations_list.html
